In [2]:
import pandas as pd
import json
import numpy as np
def load_dataset(data_dir= '/Users/kangrui/Desktop/vast/moelora/sasrec_pytorch/data/ml-1m.txt'):
    users = []
    items = []
    f = open(data_dir, 'r')
    for line in f:
        u, i = line.rstrip().split(' ')
        users.append(u)
        items.append(i)
    users = pd.Series(users)
    items = pd.Series(items)
    df = pd.DataFrame({'user': users, 'item': items})
    return df

def split_by_popularity(df: pd.DataFrame, mode='popularity'):
    ### ratio of top items (20%) in profies
    ### blockbuster - diverse - niche
    ### 54  - 26 - 20
    num_users  = len(df['user'].unique())
    num_items  = len(df['item'].unique())
    popular_items = df['item'].value_counts(ascending=False).index.to_list()[:num_items//5] ## top 20%
    def filter_helper(batch):
        all = len(batch)
        popular = len(batch[batch['item'].isin(popular_items)])
        return popular / all
    if mode == 'popularity':
        user_sorted = df.groupby('user').apply(filter_helper).sort_values(ascending = False).index.tolist()
    else:
        user_sorted = df.groupby('user')['item'].agg('count').sort_values(ascending=False).index.tolist()
    cuts  = (np.arange(1,100) * 6040 * 0.01).astype(int)
    user_groups = list()
    user_groups.append(user_sorted[:cuts[0]])
    for index, cut in enumerate(cuts):
        if index == 98:
            user_groups.append(user_sorted[cut:])
        else:
            user_groups.append(user_sorted[cut:cuts[index+1]])
    return user_groups



In [4]:
df = load_dataset()
user_sorted = df.groupby('user')['item'].agg('count').sort_values(ascending=False).index.tolist()
user_sorted[-5:]
# user_groups = split_by_popularity(df, mode='temprature')

['392', '5895', '147', '3428', '2938']

In [10]:
user_types = {}
for index, user_group in enumerate(user_groups):
    for user in user_group:
        user_types[user] = "split_" + str(index+1)


json_data = json.dumps(user_types)
# Write to a JSON file
with open('user_types_temprature.json', 'w') as json_file:
    json_file.write(json_data)

print("JSON file 'user_types.json' has been created successfully.")

JSON file 'user_types.json' has been created successfully.
